In [3]:
from dotenv import load_dotenv
load_dotenv() 

True

In [8]:
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFaceEmbeddings

In [9]:
model = 'llama3-70b-8192'

llm_groq = ChatGroq(model=model)

In [10]:
llm_groq.invoke("Program apa saja yang dipelajari di DTsense?")

AIMessage(content='DTsense adalah sebuah institusi pendidikan yang berfokus pada pengembangan kemampuan di bidang data science, artificial intelligence, dan teknologi terkait lainnya. Berikut adalah beberapa program yang dipelajari di DTsense:\n\n1. **Data Science**: Program ini membahas tentang pengolahan data, analisis data, dan visualisasi data menggunakan berbagai tools dan teknologi seperti Python, R, Tableau, Power BI, dan lain-lain.\n2. **Machine Learning**: Program ini membahas tentang pengembangan model machine learning, penggunaan algoritma machine learning, dan implementasinya dalam berbagai kasus bisnis.\n3. **Deep Learning**: Program ini membahas tentang pengembangan model deep learning, penggunaan library deep learning seperti TensorFlow, Keras, dan PyTorch, serta implementasinya dalam berbagai kasus bisnis.\n4. **Artificial Intelligence**: Program ini membahas tentang pengembangan sistem AI, penggunaan teknologi AI dalam berbagai industri, dan implementasinya dalam berba

### RAG

In [11]:
# Langkah 1: Mengambil konten dari halaman web menggunakan WebBaseLoader
url = "https://dtsense.id/ds4/"
loader = WebBaseLoader(url)
documents = loader.load()

In [12]:
documents

[Document(metadata={'source': 'https://dtsense.id/ds4/', 'title': 'Generative AI Academy – DTSense', 'language': 'en-US'}, page_content='\n\n\n\n\n\nGenerative AI Academy – DTSense\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\tSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\nHome\nAbout\n\nOur Team\nFAQ\nContact Us\n\n\nAcademy\n\nAcademy I: Data Analytics Track\n\nPendaftaran Data Analytics Track\n\n\nAcademy II: AI & Machine Learning\n\nPendaftaran Data Science II Academy\n\n\nAcademy III: Deep Learning & Reinforcement Learning\n\nPendaftaran Data Science III Academy\n\n\nGenerative AI Engineer (LLM & RAG)\nKnowledge\nAI-IDEATHON\n\n\nGenAI\nCertificate\nWorkshop\nWebinars\n \n\n Menu\n\n\nHome\nAbout\n\nOur Team\nFAQ\nContact Us\n\n\nAcademy\n\nAcademy I: Data Analytics Track\n\nPendaftaran Data Analytics Track\n\n\nAcademy II: AI & Machine Learning\n\nPendaftaran Data Science II Academy\n\n\nAcademy III: Deep L

In [ ]:
# Langkah 2: Menggunakan Hugging Face Embeddings untuk membuat vektor dari dokumen
# Menggunakan model Hugging Face untuk embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-distilroberta-v1")
faiss_index = FAISS.from_documents(documents, embeddings)

# faiss_index.save_local("doc_index" + "/")

/Users/muhammadhamdi/Documents/DTS_FEB_25/GEN_AI/introgenai_and_promt/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [14]:
faiss_index = FAISS.load_local("doc_index", embeddings, allow_dangerous_deserialization=True)

In [15]:
# Langkah 3: Menyusun pertanyaan dan melakukan pencarian berbasis kesamaan
query = "Program apa saja yang ada di DTSense?"
related_docs = faiss_index.similarity_search(query, k=3)  

In [16]:
#Menggabungkan konten dari hasil pencarian
context = "\n\n".join([doc.page_content for doc in related_docs])

# Langkah 4: Membuat PromptTemplate untuk menghasilkan respons berbasis kontekstual
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    Berdasarkan informasi berikut, jawab pertanyaan ini:\n
    Informasi:\n{context}\n
    Pertanyaan:\n{query}
    """
)

# Langkah 5: Inisialisasi model OpenAI dan membuat chain
# llm = ChatOpenAI(model_name="gpt-4o")
chain = LLMChain(llm=llm_groq, prompt=prompt_template)

# Memproses query dengan LLM untuk mendapatkan jawaban yang didukung oleh konteks dari FAISS
answer = chain.run(context=context, query=query)

/var/folders/pj/3yfymb197hv2f5g4jf72wnkm0000gn/T/ipykernel_19086/2561520368.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm_groq, prompt=prompt_template)
/var/folders/pj/3yfymb197hv2f5g4jf72wnkm0000gn/T/ipykernel_19086/2561520368.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain.run(context=context, query=query)


In [17]:
answer

'Berikut adalah program-program yang ada di DTSense:\n\n1. Generative AI Academy\n2. Generative AI Engineer Career Fast-Track\n3. Academy I: Data Analytics Track\n4. Academy II: AI & Machine Learning\n5. Academy III: Deep Learning & Reinforcement Learning'

In [18]:
chain.run(context=context, query='Apa yang dipelajari pada hari ke 5?')

'Pada hari ke 5, peserta akan mempelajari "Introduction to GenAI & Prompt Engineering (LLM Models, e.g: GPT3.5, GPT4o, LLAMA2,LLAMA3, DBRX)".'

In [19]:
chain.run(context=context, query='Apa kelebihan career track?')

'Berdasarkan informasi yang diberikan, kelebihan dari Generative AI Engineer Career Fast-Track adalah:\n\n* Peserta akan belajar materi yang ada di Generative AI Academy ditambah dengan kelas tambahan seperti Data Analytics, AI Engineering, Advanced LLM topic, dan juga sesi untuk support Karir/Akademik/Beasiswa untuk ke Luar Negeri, khususnya di Eropa.\n* Peserta yang lulus dalam program ini bisa lebih siap untuk berkarir menjadi GenAI Engineer di dalam Negeri maupun di luar Negeri.\n* Estimasi gaji GenAI Engineer di United States: $130,518 / ~2.1 Milyar Rupiah dan di Eropa - Jerman: €71,060 / ~1.25 Milyar Rupiah.\n\nDengan demikian, kelebihan dari career track ini adalah peserta akan mendapatkan pengetahuan dan keterampilan yang lebih luas dan mendalam, serta memiliki peluang yang lebih baik untuk berkarir di bidang GenAI Engineer dengan gaji yang lebih tinggi.'